## Exercise 1: Prompt Engineering

Let's consider LLAMA as our starting point. In the following, we see a typical prompt feeding and text generation with LLAMA

In [2]:
from huggingface_hub import login

with open("hf_token.txt", "r") as f:
    token = f.read()
    f.close()

login(token=token)

In [3]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Assuming model and tokenizer are already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device (GPU if available)
model = model.to(device)


# Input prompt - Make it clear that you want only the direct answer without any explanations or options
prompt = """
System: You are an expert on world capitals.
Respond with only the capital city of the given country. Do not repeat the question.

Query: What is the capital of France?
Answer:
"""

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate a response
output = model.generate(
    inputs['input_ids'],  # Tokenized input
    max_length=100,         # Limit response length to avoid extra text
    temperature=0.7,        # Lower temperature to reduce randomness
    do_sample=True,        # Disable sampling for deterministic output
    pad_token_id=tokenizer.eos_token_id  # Ensure the model doesn't go beyond the end token

)

# Decode the response into human-readable text
response = tokenizer.decode(output[0], skip_special_tokens=True)

answer = response.split("query:")[-1].strip()
print("Response:", answer)



config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response: System: You are an expert on world capitals.
Respond with only the capital city of the given country. Do not repeat the question.

Query: What is the capital of France?
Answer:
Paris


### Fitz

Reference libraries to install: pip install openai pymupdf faiss-cpu scikit-learn

PyMuPDF is a Python library that **provides tools for working with PDF files** (as well as other document formats like XPS, OpenXPS, CBZ, EPUB, and FB2). It's built on the MuPDF library, a lightweight, high-performance PDF and XPS rendering engine. With PyMuPDF, you can perform various tasks like reading, creating, editing, and extracting content from PDFs, images, and annotations.

In [5]:
!pip install openai pymupdf faiss-cpu scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.8 MB/s eta 0:00:00


In [7]:
import fitz

#open an example pdf
doc = fitz.open("example2.pdf")

# Extract text from the first page
page = doc.load_page(0)
text = page.get_text("text")  # Use 'text' mode to get raw text
print(text)


Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 1926. The stadium, which was 
built by Milan's second chairman, Piero Pirelli and has been shared with Inter Milan since 1947, is the 
largest in Italian football, with a total capacity of 75,817. The club has a long-standing rivalry with Inter, 
with whom they contest the Derby della Madonnina, one of the most followed derbies in football. 
 
Milan has spent its entire history in Serie A with the exception of the 1980–81 and 1982–83 seasons. 
Silvio Berlusconi’s 31-year tenure as Milan president was a standout period in the club's history, as 
they established themselves as one of Europe's most dominant and successful

In [13]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text+=page.get_text()
    return text

### Example: Text Summarization

Let's ask LLAMA to perform a summarization of the example PDF.

In [18]:
#define the prompt to ask for text summarization.

#define your prompt here
text_summarization_prompt = """
Write a concise summary of the main ideas in article below.
"""
#load here the FULL text of the article
text = extract_text_from_pdf("example2.pdf")
p1 =  """{PROMPT}. article: {BODY} \nAnswer:""".format(PROMPT=text_summarization_prompt, BODY=text)

#feed the prompt to llama
#print the result of text summarization into bullets

# Tokenize the input
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p1, return_tensors="pt", padding=True, truncation=True).to('cuda')

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r1 = tokenizer.decode(output[0], skip_special_tokens=True)
print(r1)


Write a concise summary of the main ideas in article below.
. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 1926. The stadium, which was 
built by Milan's second chairman, Piero Pirelli and has been shared with Inter Milan since 1947, is the 
largest in Italian football, with a total capacity of 75,817. The club has a long-standing rivalry with Inter, 
with whom they contest the Derby della Madonnina, one of the most followed derbies in football. 
 
Milan has spent its entire history in Serie A with the exception of the 1980–81 and 1982–83 seasons. 
Silvio Berlusconi’s 31-year tenure as Milan president was a standout period in the club's history, as 
the

### Adding a System Prompt

Llama was trained with a system message that set the context and persona to assume when solving a task. One of the unsung advantages of open-access models is that you have full control over the system prompt in chat applications. This is essential to specify the behavior of your chat assistant –and even imbue it with some personality–, but it's unreachable in models served behind APIs.


In [21]:
#default standard system message from the Hugging Face blog to the prompt from above
system_prompt = "<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information. <</SYS>>"

#concatenate the system prompt with your pront and get the response
p2 =  """{SYS}\nQuestion: {PROMPT}. article: {BODY} \nAnswer:""".format(SYS=system_prompt, PROMPT=text_summarization_prompt, BODY=text)

# Tokenize the input
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p2, return_tensors="pt", padding=True, truncation=True).to('cuda')

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r2 = tokenizer.decode(output[0], skip_special_tokens=True)
print(r2)

#what changes?

<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information. <</SYS>>
Question: 
Write a concise summary of the main ideas in article below.
. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 

### Customizing the System prompt

With Llama we have full control over the system prompt. The following experiment will instruct Llama to assume the persona of a researcher tasked with writing a concise brief.

Apply the following changes the original system prompt:
- Use the researcher persona and specify the tasks to summarize articles.
- Remove safety instructions; they are unnecessary since we ask Llama to be truthful to the article.


In [22]:
new_system_prompt = "<<SYS>> You are a helpful, respectful and honest assistant. \
    Assume that you are a researcher tasked with writing a concise brief. <</SYS>>"

#concatenate the system prompt with your pront and get the response
p3 =  """{SYS}\nQuestion: {PROMPT}. article: {BODY} \nAnswer:""".format(SYS=system_prompt, PROMPT=text_summarization_prompt, BODY=text)

# Tokenize the input
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p3, return_tensors="pt", padding=True, truncation=True).to('cuda')

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r3 = tokenizer.decode(output[0], skip_special_tokens=True)
print(r3)

<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information. <</SYS>>
Question: 
Write a concise summary of the main ideas in article below.
. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 

PROFESSOR PROPOSAL:

In [23]:
new_system_prompt = """<<SYS>> You are a helpful assistant.
    When summarizing an article or providing key points, you must:
    1. Use **bullet points** to list the main ideas.
    2. **Bullet points must be formatted like this:**
       - First key point goes here.
       - Second key point goes here.
       - Third key point goes here.
    3. Each bullet point should be one short sentence.
    4. Keep bullet points clear, concise, and easy to read.
    5. Make sure every distinct idea from the article is listed as a separate bullet point.
    6. Avoid paragraphs; only bullet points should be used in the output.
<</SYS>>"""

text_summarization_prompt2 = "Summarize the article below into bullet points. Format bullet points exactly as described above"

p3 =  """{SYS}\nQuestion: {PROMPT}. article: {BODY} \nAnswer (use bullet points exactly like this: - point 1):""".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt2, BODY=text)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p3, return_tensors="pt", padding=True, truncation=True).to(device)



# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=500,
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r3 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r3)

Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful assistant. 
    When summarizing an article or providing key points, you must:
    1. Use **bullet points** to list the main ideas.
    2. **Bullet points must be formatted like this:**
       - First key point goes here.
       - Second key point goes here.
       - Third key point goes here.
    3. Each bullet point should be one short sentence.
    4. Keep bullet points clear, concise, and easy to read.
    5. Make sure every distinct idea from the article is listed as a separate bullet point.
    6. Avoid paragraphs; only bullet points should be used in the output.
<</SYS>>
Question: Summarize the article below into bullet points. Format bullet points exactly as described above. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its hom

### Chain-of-Thought prompting

Chain-of-thought is when a prompt is being constructed using a previous prompt answer. For our use case to extract information from text, we will first ask Llama what the article is about and then use the response to ask a second question: what problem does [what the article is about] solve?



In [26]:
#define a prompt to ask what the article is about

start_phrase = "Answer (use bullet points exactly like this: - point 1):"
r4 = r3.split(start_phrase, 1)[1].strip()

#now embed the result of the previous prompt in a new prompt to ask what that solves
question = "what problem does [what the article is about] solve?"

p5 =  """{SYS}\nQuestion: {PROMPT}. article: {BODY} \nAnswer:""".format(SYS=new_system_prompt, PROMPT=question, BODY=r4)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p5, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=500,
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r5 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r5)




Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful assistant. 
    When summarizing an article or providing key points, you must:
    1. Use **bullet points** to list the main ideas.
    2. **Bullet points must be formatted like this:**
       - First key point goes here.
       - Second key point goes here.
       - Third key point goes here.
    3. Each bullet point should be one short sentence.
    4. Keep bullet points clear, concise, and easy to read.
    5. Make sure every distinct idea from the article is listed as a separate bullet point.
    6. Avoid paragraphs; only bullet points should be used in the output.
<</SYS>>
Question: what problem does [what the article is about] solve?. article: - Point 1: Milan is a professional football club based in Milan, Italy. Founded in 1899, the club competes in the Serie A, the top tier of Italian football. In its early history, Milan played its home games in different grounds around the city before moving to its current stadium, the San Siro, in 1926. The stadium

### Generating JSONs with Llama

Llama needs precise instructions when asking it to generate JSON. In essence, here is what works for me to get valid JSON consistently:

- Explicitly state — “ All output must be in valid JSON. Don’t add explanation beyond the JSON” in the system prompt.
- Add an “explanation” variable to the JSON example. Llama enjoys explaining its answers. Give it an outlet.
- Use the JSON as part of the instruction. See the “in_less_than_ten_words” example below.
Change “write the answer” to “output the answer.”


In [27]:


#example addition to a prompt to deal with jsons
json_prompt_addition = "Output must be in valid JSON like the following example {{\"sport\": sport, \"explanation\": [in_less_than_ten_words]}}. Output must include only JSON."

new_system_prompt = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
    Provide straight to the point answers, without bullet points.
    Output must be in valid JSON like the following example {{\"sport\": sport, \"explanation\": [in_less_than_ten_words]}}. Output must include only JSON.
<</SYS>>"""

text_summarization_prompt3 = "Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different aspects"

#now generate a prompt by correctly concatenating the system prompt, the json prompt instruction, and a sports article
p6 =  """{SYS}\nQuestion: {PROMPT}. \narticle: {BODY} \nAnswer:""".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt3, BODY=text)

print(p6)

inputs = tokenizer(p6, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=500,
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r6 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r6)

#compare the difference between the prompt with the formatting instruction and a regular prompt without formatting instructions. is there any difference?

Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
    Provide straight to the point answers, without bullet points.
    Output must be in valid JSON like the following example {{"sport": sport, "explanation": [in_less_than_ten_words]}}. Output must include only JSON.
<</SYS>>
Question: Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different aspects. 
article: Associazione Calcio Milan, commonly referred to as AC Milan

### One-to-Many Shot Learning Prompting

One-to-Many Shot Learning is a term that refers to a type of machine learning problem where the goal is to learn to recognize many different classes of objects from only one or a few examples of each class. For example, if you have only one image of a cat and one image of a dog, can you train a model to distinguish between cats and dogs in new images? This is a challenging problem because the model has to generalize well from minimal data (source)

Important points about the prompts:

- The system prompt includes the instructions to output the answer in JSON.
- The prompt consists of an one-to-many shot learning section that starts after ```<</SYS>>``` and ends with ```</s>```.  See the prompt template below will make it easier to understand.
- The examples are given in JSON because the answers need to be JSON.
- The JSON allows defining the response with name, type, and explanation.
- The prompt question start with the second ```<s>[INST]``` and end with the last ```[/INST]```

```
<s>[INST] <<SYS>>
SYSTEM MESSAGE
<</SYS>>
EXAMPLE QUESTION [/INST]
EXAMPLE ANSWER(S)
</s>
<s>[INST]  
QUESTION
[/INST]
```

In [29]:
#describe all the main nouns in the example.pdf article

#use the following addition for one-to-many prompting exampling
nouns = """[\
{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},\
{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},\
{{"name": "Ford F150", "type": "vehicle", "explanation": "Article talks about the Ford F150 truck"}},\
{{"name": "Ford", "type": "company", "explanation": "Ford is a company that built vehicles"}},\
{{"name": "John Smith", "type": "person", "explanation": "Mentioned in the article"}},\
]"""

new_system_prompt = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
    Provide straight to the point answers, without bullet points.
    Output must be in the following format, by describing names and descriptions extracted by the article = \
    """ + nouns + """
<</SYS>>"""

text_summarization_prompt3 = "Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different info about Milan."

#now build the prompt following the template described above
p7 =  """{SYS}\nQuestion: {PROMPT}. \narticle: {BODY} \nAnswer:""".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt3, BODY=text)

#print(p7)

inputs = tokenizer(p7, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=500,
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r7 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r7)

#compare the response of the prompt described above and a zero-shot prompt. Are there any differences?


Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
    Provide straight to the point answers, without bullet points.
    Output must be in the following format, by describing names and descriptions extracted by the article =     [{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},{{"name": "Ford F150", "type":

## Exercise 2: RAG (Retrieval-Augmented-Generation)

RAG (Retrieval-Augmented Generation) is a powerful framework in Natural Language Processing (NLP) that enhances the performance of language models by combining traditional generative models with external knowledge retrieval. This hybrid approach allows models to retrieve relevant information from a large corpus (like a database or document collection) and incorporate this information into the generation process. It is particularly useful when a model needs to answer questions, generate content, or provide explanations based on real-time or domain-specific data.



In [ ]:
import os
import glob


#TODO:  Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    print("")
    #your code here...

# Extract text from all uploaded PDF files
pdf_texts = {}
# your code here...

#Display the text from all the PDF files
for pdf_file, text in pdf_texts.items():
    print("") #implement PDF read

### Creating an index of vectors to represent the documents

To perform efficient searches, we need to convert our text data into numerical vectors. To do so, we will use the first step of the BERT transformer.

Since our full pdf files are very long to be fed as input into BERT, we perform a step in which we create a structure where we associate a document number to its abstract, and in a separate dictionary we associate a document number to its full text.


In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np




#import the Bert pretrained model from the transformers library
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#initialization of the dictionary of abstracts. Substitute this with the abstracts of the 10 papers considered as sources for RAG
#(we could use functions to read the PDFs to "cut" the abstracts from the papers. For simplicity reasons, we will copy and paste them)
abstracts_dict = {
    0: ""
}

#the text for rag is used as an input to the BERT model

#The tokenized inputs are passed to the BERT model for processing.
#(#remember padding=True: Ensures that all inputs are padded to the same length, allowing batch processing.)
#The model outputs a tensor (last_hidden_state), where each input token is represented by a high-dimensional vector.
#last_hidden_state is of shape (batch_size, sequence_length, hidden_size), where:
#batch_size: Number of input texts.
#sequence_length: Length of each tokenized text (after padding).
#hidden_size: Dimensionality of the vector representation for each token (default 768 for bert-base-uncased).

#last_hidden_state[:, 0]: Selects the representation of the [CLS] token for each input text. The [CLS] token is a special token added at the start of each input and is often used as the aggregate representation for the entire sequence.

abstract_vectors = ""

#abstract_vectors is a tensor of shape (batch_size, hidden_size) (e.g., (3, 768) in this case), representing each text as a single 768-dimensional vector.

print(abstract_vectors.shape)



### Search

With our text data vectorized and indexed, we can now perform searches. We will define a function to search the index for the most relevant documents based on a query.

To perform the search, we need a function (search documents) where we perform the cosine similarity between the query vector and all the abstract vectors. This function will give our the top-k indexes. Once we find the top-k indexes, with another function, we can collect the full text of the documents from the paper dictionary.

To compute cosine similarity, refer to the following formula

```cs = cosine_similarity(vector_a.detach().numpy(), vector_b.detach().numpy())```



In [ ]:




def get_top_k_similar_indices(query_vector, abstract_vectors, k):

    #Computes the top k indices of the most similar abstracts to the query based on cosine similarity.

    #Parameters:
    #- query_vector: A tensor of shape (1, hidden_size) representing the query vector.
    #- abstract_vectors: A tensor of shape (batch_size, hidden_size) representing the abstract vectors.
    #- k: The number of top indices to return.

    #Returns:
    #- sorted_indices: A numpy array of shape (1, k) containing the indices of the top k most similar abstracts.

    return ""


def retrieve_documents(indices, documents_dict):

    #Retrieves the documents corresponding to the given indices and concatenates them into a single string.

    #Parameters:
    #- indices: A numpy array or list of top-k indices of the most similar documents.
    #- documents_dict: A dictionary where keys are document indices (integers) and values are the document texts (strings).

    #Returns:
    #- concatenated_documents: A string containing the concatenated texts of the retrieved documents.

    return ""



#now I create a vector also for my query

query = ""

query_vector = ""





### A function to perform Retrieval Augmented Generation

In this step, we’ll combine the context retrieved from our documents with LLAMA to generate responses. The context will provide the necessary information to the model to produce more accurate and relevant answers.

In [ ]:


#now we put it all together

def generate_augmented_response(query, documents):

    system = ""             #TODO: define system prompt

    context = ""               #TODO: concatenate here all the search results


    prompt = ""                 #TODO: create the prompt for LLAMA (system + context + query)

    response = ""

    #perform a query with LLAMA in the usual way

    #return the response
    return ""


# TODO: generate the queries!
query = ""
response = generate_augmented_response(query)
print(response)

#TODO: now compare the results with a prompt without RAG. What are the results?
